# 单卡GPU 进行 ChatGLM3-6B模型 LORA 高效微调
本 Cookbook 将带领开发者使用 `AdvertiseGen` 对 ChatGLM3-6B 数据集进行 lora微调，使其具备专业的广告生成能力。

## 硬件需求
显存：24GB
显卡架构：安培架构（推荐）
内存：16GB

## 1. 准备数据集
我们使用 AdvertiseGen 数据集来进行微调。从 [Google Drive](https://drive.google.com/file/d/13_vf0xRTQsyneRKdD1bZIr93vBGOczrk/view?usp=sharing) 或者 [Tsinghua Cloud](https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1) 下载处理好的 AdvertiseGen 数据集，将解压后的 AdvertiseGen 目录放到本目录的 `/data/` 下, 例如。
> /media/zr/Data/Code/ChatGLM3/finetune_demo/data/AdvertiseGen

接着，运行本代码来切割数据集

In [2]:
from pathlib import Path
from combine_data import combine_data, analysis_data

print(Path.cwd())

# combine_data(Path('./data/raw'), Path('./data/fix'))

analysis_data(Path('./data/fix'))


/home/cike/HJS/funcCall/ChatGLM3-main/finetune_demo
共15932, val1593.2, test1593.2, train12745.6


## 2. 使用命令行开始微调,我们使用 lora 进行微调
接着，我们仅需要将配置好的参数以命令行的形式传参给程序，就可以使用命令行进行高效微调，这里将 `/media/zr/Data/Code/ChatGLM3/venv/bin/python3` 换成你的 python3 的绝对路径以保证正常运行。

In [4]:
!CUDA_VISIBLE_DEVICES=1 /home/cike/miniconda3/envs/wiseSight/bin/python3 finetune_hf.py ./data/fix /home/cike/glm_and_vlm/chatglm3-6b  configs/lora.yaml

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
2024-04-01 00:28:08.721087: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 00:28:08.759218: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 00:28:08.759258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 00:28:08.760213: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been register

## 3. 使用微调的数据集进行推理
在完成微调任务之后，我们可以查看到 `output` 文件夹下多了很多个`checkpoint-*`的文件夹，这些文件夹代表了训练的轮数。
我们选择最后一轮的微调权重，并使用inference进行导入。

In [6]:
!ls output/

checkpoint-1000  checkpoint-2000  checkpoint-3000
checkpoint-1500  checkpoint-2500  checkpoint-500


In [9]:
!CUDA_VISIBLE_DEVICES=1 /home/cike/miniconda3/envs/wiseSight/bin/python3 inference_hf.py ./output/checkpoint-3000/ --prompt "今天天气如何"

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:24<00:00,  3.50s/it]
这款网纱拼接的连衣裙，采用不规则的木耳边压褶设计，打造出了个性十足的不规则裙摆，轻松穿出时尚性感。腰部的木耳边拉链套头设计，方便穿脱，又可以修饰腰部线条，显得更加精致。下摆采用百褶设计，修饰腿型，显瘦又显高。


## 4. 总结
到此位置，我们就完成了使用单张 GPU Lora 来微调 ChatGLM3-6B 模型，使其能生产出更好的广告。
在本章节中，你将会学会：
+ 如何使用模型进行 Lora 微调
+ 微调数据集的准备和对齐
+ 使用微调的模型进行推理